In [ ]:
/content/sample_data

In [1]:
pip install scikit-posthocs

In [3]:

import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp
import csv

#  CONFIG
INPUT = "/content/sample_data/cycling.txt"
OUTDIR = "cycling_analysis_outputs"
Path(OUTDIR).mkdir(exist_ok=True)
sns.set(style="whitegrid")

def load_cycling(path):
    """
    Robust loader for the provided file format:
    - fields are double-quoted and separated by spaces
    - columns: all_riders, rider_class, stage, points, stage_class
    """
    try:
        df = pd.read_csv(path, sep=r'\s+', engine='python', quotechar='"', header=0)
        if df.shape[1] != 5:

            rows = []
            with open(path, newline='', encoding='utf-8') as f:
                reader = csv.reader(f, delimiter=' ', quotechar='"', skipinitialspace=True)
                for r in reader:

                    tokens = [t for t in r if t != '']
                    if tokens:
                        rows.append(tokens)
            df = pd.DataFrame(rows[1:], columns=[c.strip().strip('"') for c in rows[0]])
    except Exception:

        rows = []
        with open(path, newline='', encoding='utf-8') as f:
            reader = csv.reader(f, delimiter=' ', quotechar='"', skipinitialspace=True)
            for r in reader:
                tokens = [t for t in r if t != '']
                if tokens:
                    rows.append(tokens)
        df = pd.DataFrame(rows[1:], columns=[c.strip().strip('"') for c in rows[0]])

    # Normalize column names
    df.columns = [c.strip().strip('"').lower() for c in df.columns]
    return df

# Load
df = load_cycling(INPUT)
print("Loaded shape:", df.shape)
print("Columns:", df.columns.tolist())
print(df.head(6))


expected = {'all_riders', 'rider_class', 'stage', 'points', 'stage_class'}
missing = expected - set(df.columns)
if missing:
    raise RuntimeError(f"Missing expected columns: {missing}. Detected columns: {df.columns.tolist()}")


df = df.rename(columns={'all_riders': 'rider', 'rider_class': 'rclass'})

df['points'] = pd.to_numeric(df['points'], errors='coerce')
df = df.dropna(subset=['points'])

df['rider'] = df['rider'].astype(str).str.strip().str.replace('"','')
df['rclass'] = df['rclass'].astype(str).str.strip().str.title()
df['stage'] = df['stage'].astype(str).str.strip()
df['stage_class'] = df['stage_class'].astype(str).str.strip().str.lower()


df['stage_class'] = df['stage_class'].replace({'mount': 'mountain', 'mount.': 'mountain'})


print("\nUnique rider classes:", sorted(df['rclass'].unique()))
print("Unique stage classes:", sorted(df['stage_class'].unique()))
print("Total rows after cleaning:", len(df))

# ---------------- Descriptive statistics ----------------
def desc_table(group_cols):
    g = df.groupby(group_cols)['points']
    q1 = g.quantile(0.25)
    q3 = g.quantile(0.75)
    res = g.agg(['count', 'mean', 'median', 'std', 'min', 'max']).join(q1.rename('q1')).join(q3.rename('q3'))
    res['iqr'] = res['q3'] - res['q1']
    res = res.reset_index()
    return res

desc_rclass = desc_table(['rclass'])
desc_stageclass = desc_table(['stage_class'])
desc_combined = desc_table(['rclass','stage_class'])

desc_rclass.to_csv(f"{OUTDIR}/desc_by_rclass.csv", index=False)
desc_stageclass.to_csv(f"{OUTDIR}/desc_by_stage_class.csv", index=False)
desc_combined.to_csv(f"{OUTDIR}/desc_by_rclass_stageclass.csv", index=False)



plt.figure(figsize=(9,6))
order = sorted(df['rclass'].unique())
ax = sns.boxplot(x='rclass', y='points', data=df, order=order)
ax.set_title("Points by Rider Class")
ax.set_xlabel("Rider class")
ax.set_ylabel("Points")
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig(f"{OUTDIR}/boxpoints_by_rclass.png", dpi=300)
plt.close()


g = sns.catplot(x='rclass', y='points', col='stage_class', data=df, kind='box',
                col_order=sorted(df['stage_class'].unique()), sharey=True, height=5, aspect=0.95)
g.set_axis_labels("Rider class", "Points")
g.fig.suptitle("Points by Rider Class, separated by Stage Class", y=1.02)
for ax in g.axes.flatten():
    for label in ax.get_xticklabels():
        label.set_rotation(12)
plt.tight_layout()
g.savefig(f"{OUTDIR}/boxpoints_by_rclass_by_stageclass.png", dpi=300)
plt.close()


means = df.groupby(['rclass','stage_class'])['points'].mean().reset_index()
plt.figure(figsize=(9,6))
for cls in sorted(df['rclass'].unique()):
    subset = means[means['rclass']==cls].sort_values('stage_class')
    plt.plot(subset['stage_class'], subset['points'], marker='o', label=cls)
plt.xlabel('Stage class')
plt.ylabel('Mean points')
plt.title('Interaction plot: mean points by rider class and stage class')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(f"{OUTDIR}/interaction_mean_points.png", dpi=300)
plt.close()


grouped = df.groupby(['rclass','stage_class'])
shapiro_results = []
for name, group in grouped:
    n = len(group)
    if n >= 3 and n <= 5000:
        try:
            stat, p = stats.shapiro(group['points'])
        except Exception:
            stat, p = np.nan, np.nan
    else:
        stat, p = np.nan, np.nan
    shapiro_results.append({'rclass':name[0], 'stage_class':name[1], 'n':n,'shapiro_stat':stat,'shapiro_p':p})
shap = pd.DataFrame(shapiro_results)
shap.to_csv(f"{OUTDIR}/shapiro_by_group.csv", index=False)


df['group_label'] = df['rclass'] + " | " + df['stage_class']
group_vals = [df.loc[df['group_label']==g, 'points'].values for g in df['group_label'].unique()]

group_vals = [g for g in group_vals if len(g) >= 2]
if len(group_vals) >= 2:
    levene_stat, levene_p = stats.levene(*group_vals, center='median')
else:
    levene_stat, levene_p = np.nan, np.nan

with open(f"{OUTDIR}/assumption_results.txt", 'w') as f:
    f.write(f"Levene stat (median-centered): {levene_stat}, p = {levene_p}\n")
    f.write("Shapiro-Wilk per-group: shapiro_by_group.csv\n")


formula = 'points ~ C(rclass) * C(stage_class)'
model = ols(formula, data=df).fit()
anova_res = sm.stats.anova_lm(model, typ=2)
anova_res.to_csv(f"{OUTDIR}/two_way_anova_type2.csv")
print("\nANOVA results:\n", anova_res)


try:
    tukey_rclass = pairwise_tukeyhsd(df['points'], df['rclass'])
    with open(f"{OUTDIR}/tukey_rclass.txt","w") as f:
        f.write(str(tukey_rclass.summary()))
except Exception as e:
    print("Tukey for rclass failed:", e)


df['combo'] = df['rclass'] + "_" + df['stage_class']
if df['combo'].nunique() <= 40:
    try:
        tukey_combo = pairwise_tukeyhsd(df['points'], df['combo'])
        with open(f"{OUTDIR}/tukey_combo.txt","w") as f:
            f.write(str(tukey_combo.summary()))
    except Exception as e:
        print("Tukey combo failed:", e)


kw_results = []
for sc in sorted(df['stage_class'].unique()):
    sub = df[df['stage_class']==sc]
    groups = [sub.loc[sub['rclass']==r,'points'].values for r in sorted(sub['rclass'].unique())]

    groups = [g for g in groups if len(g) > 0]
    if len(groups) >= 2:
        try:
            stat, p = stats.kruskal(*groups)
        except Exception:
            stat, p = np.nan, np.nan
    else:
        stat, p = np.nan, np.nan
    kw_results.append({'stage_class': sc, 'kw_stat': stat, 'kw_p': p})
pd.DataFrame(kw_results).to_csv(f"{OUTDIR}/kruskal_by_stage_class.csv", index=False)


for sc in sorted(df['stage_class'].unique()):
    sub = df[df['stage_class']==sc]
    if sub['rclass'].nunique() >= 2:
        pvals = sp.posthoc_dunn(sub, val_col='points', group_col='rclass', p_adjust='holm')
        pvals.to_csv(f"{OUTDIR}/dunn_{sc}.csv")


df.to_csv(f"{OUTDIR}/cycling_cleaned.csv", index=False)

with open(f"{OUTDIR}/results_summary.txt", 'w') as f:
    f.write("Summary of analysis\n")
    f.write("-------------------\n")
    f.write("Descriptive stats saved: desc_by_rclass.csv, desc_by_stage_class.csv, desc_by_rclass_stageclass.csv\n")
    f.write("ANOVA results saved: two_way_anova_type2.csv\n")
    f.write(f"Levene (median) p-value: {levene_p}\n")
    f.write("Shapiro per-group saved to shapiro_by_group.csv\n")
    f.write("If assumptions violated, use Kruskal-Wallis (kruskal_by_stage_class.csv) and Dunn pairwise (dunn_*.csv)\n")
    f.write("Figures saved: boxpoints_by_rclass.png, boxpoints_by_rclass_by_stageclass.png, interaction_mean_points.png\n")

print("\nOutputs saved to:", OUTDIR)


Loaded shape: (3496, 5)
Columns: ['all_riders', 'rider_class', 'stage', 'points', 'stage_class']
      all_riders  rider_class stage points stage_class
0  Tadej Pogačar  All Rounder    X1     15        flat
1  Tadej Pogačar  All Rounder    X2    219       hills
2  Tadej Pogačar  All Rounder    X3     34        flat
3  Tadej Pogačar  All Rounder    X4    264       hills
4  Tadej Pogačar  All Rounder    X6    114       hills
5  Tadej Pogačar  All Rounder    X7    274       hills

Unique rider classes: ['All Rounder', 'Climber', 'Sprinter', 'Unclassed']
Unique stage classes: ['flat', 'hills', 'mountain']
Total rows after cleaning: 3496

ANOVA results:
                                 sum_sq      df          F        PR(>F)
C(rclass)                 3.148937e+05     3.0  92.816103  8.703760e-58
C(stage_class)            6.359253e+02     2.0   0.281162  7.549231e-01
C(rclass):C(stage_class)  3.460646e+05     6.0  51.001926  2.041749e-60
Residual                  3.940012e+06  3484.0        

In [4]:
print(df.head())

           rider       rclass stage  points stage_class          group_label  \
0  Tadej Pogačar  All Rounder    X1      15        flat   All Rounder | flat   
1  Tadej Pogačar  All Rounder    X2     219       hills  All Rounder | hills   
2  Tadej Pogačar  All Rounder    X3      34        flat   All Rounder | flat   
3  Tadej Pogačar  All Rounder    X4     264       hills  All Rounder | hills   
4  Tadej Pogačar  All Rounder    X6     114       hills  All Rounder | hills   

               combo  
0   All Rounder_flat  
1  All Rounder_hills  
2   All Rounder_flat  
3  All Rounder_hills  
4  All Rounder_hills  
